# Smart EDGAR: Calculation of Growth Parameters

Financial KPIs can be used to drive investment decisions. So it was my goal to create a comprehensive set of KPIs across different __dimensions__ that are based on the information which can be determined from EDGAR: 
- Profitibility
- Liquidity
- Efficiency
- Innovation
- Growth
- Leadereship
- Surprises

In this document we demonstrate the approach on how to calculate the __Growth Parameters__

In [1]:
%classpath config resolver maven-public http://software.pschatzmann.ch/repository/maven-public/
%%classpath add mvn 
ch.pschatzmann:smart-edgar:1.0.2
ch.pschatzmann:jupyter-jdk-extensions:1.0.0


Added new repo: maven-public


In [2]:
import ch.pschatzmann.edgar.reporting.company._
import ch.pschatzmann.display.Displayers

Displayers.setup

true

## Comparision with Filing from Last Year
The simplest approach to determine Growth is to use the filing from 1 year ago to calculate the percent difference. So we compare the reported cumulated values of
- Q1 vs Q1 of previous Year (for 3 months)
- Q2 vs Q2 of previous Year (for 6 months)
- Q3 vs Q3 of previous Year (for 9 months)
- Q4 vs Q4 of previous Year (for 1 year)

The calculation can be done with the method percentChangeMonths: We subtract 12 months to get the values from the previous year!

In [3]:
val values = new CompanyEdgarValuesDB(new CompanySelection().setTradingSymbol("AAPL"))
    .setAddTime(true)
    .setFilter(new FilterQuarterlyCumulated())
    .setParameterNames("NetIncomeLoss", "Revenues","SalesRevenueNet","EarningsPerShareBasic")
    .addFormula("Revenue","Edgar.coalesce('Revenues', 'SalesRevenueNet')")
    .addFormula("NetIncomeLossIncrease%","Edgar.percentChangeMonths('NetIncomeLoss',-12)")
    .addFormula("RevenueIncrease%","Edgar.percentChangeMonths('Revenue',-12)")
    .addFormula("EarningsPerShareIncrease%","Edgar.percentChangeMonths('EarningsPerShareBasic',-12)")
    .removeParameterNames("Revenues","SalesRevenueNet")

values.getTable

We convert the data to Scala as a preparation so that we can create some charts

In [4]:
import scala.collection.JavaConverters._

val scalaValues = values.toList.asScala
    .map(map => map.asScala.toMap)
    .toSeq

scalaValues.size

45

In [5]:
val plot = new SimpleTimePlot {
    data = scalaValues
    columns = Seq("RevenueIncrease%","NetIncomeLossIncrease%","EarningsPerShareIncrease%")
}
